In [2]:
# ========================================
# 📌 1. Instalación de dependencias
# ========================================
!pip install langchain-community faiss-cpu pypdf python-docx unstructured -q
!pip install msoffcrypto-tool
# Instalar la librería jq necesaria para JSONLoader
!pip install jq -q

import json
import importlib.metadata    # Importación del modulo de metadata de la libreria importlib
import warnings               # Importación de la libreria warnings

# Uso de la libreria importlib.metadata para obtener los paquetes
# instalados en el Notebook
installed_packages = [dist.metadata['Name'].lower() for dist in importlib.metadata.Distribution.discover()]

# Chequeando que Google Colab este entre los paquetes instalados
# para confirmar si es el ambiente de trabajo usado.
IN_COLAB = 'google-colab' in installed_packages

# Ignorar algun Warning generado durante la ejecución del Notebook
warnings.filterwarnings('ignore')

# Instalar Ollama solo si no está ya instalado
!if ! type ollama > /dev/null; then curl -fsSL https://ollama.com/install.sh | sh; else echo "Ollama ya está instalado."; fi

# Iniciar Ollama en segundo plano (con logs en ollama.log)
!nohup ollama serve > ollama.log 2>&1 &

# Descargar los modelos necesarios
!ollama pull llama3.2:3b
!ollama pull nomic-embed-text

Ollama ya está instalado.




In [4]:
# ========================================
# 📌 2. Carga de Documentos
# ========================================
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredWordDocumentLoader, UnstructuredExcelLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import drive
from langchain.schema import Document # Import Document class


pdf_loader = PyPDFLoader('/content/drive/MyDrive/Ecomarket/Política de Devoluciones.pdf') # ruta de archivo PDF
ruta_json = '/content/drive/MyDrive/Ecomarket/faq.json' # ruta archivo JSON
# Cargar archivo JSON. Usando jq_schema='.' para cargar el contenido completo.
# Si el JSON es una lista de objetos.
# Usamos text_content=False para evitar que JSONLoader valide si el contenido es string al cargar.
json_loader = JSONLoader(file_path=ruta_json, jq_schema='.', text_content=False)
doc_loader = UnstructuredExcelLoader('/content/drive/MyDrive/Ecomarket/Inventario_Sostenible.xlsx') # ruta de archivo Excel

# Cargar documentos de cada fuente
pdf_docs = pdf_loader.load()
excel_docs = doc_loader.load() # renombrado de docx_loader ya que ahora carga excel

# Cargar JSON y manejar si devuelve una lista
json_output = json_loader.load()
json_docs = []

# Asegurarse de procesar json_output como una lista de elementos para crear documentos.
# Incluso si jq_schema='.' devuelve un solo item, se trata como una lista de un elemento.
if not isinstance(json_output, list):
    json_output = [json_output] # Convertir a lista si no lo es

for item in json_output:
    # Convertir cada item a cadena para el page_content
    content = str(item)
    json_docs.append(Document(page_content=content, metadata={"source": ruta_json}))


docs = pdf_docs + json_docs + excel_docs

# Segmentación en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(f"Documentos cargados y segmentados en {len(chunks)} chunks")

Documentos cargados y segmentados en 10 chunks


In [6]:
# ========================================
# 📌 3. Embeddings + Base de Datos Vectorial
# ========================================


from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# Crear embeddings con Ollama
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Indexar en FAISS
db = FAISS.from_documents(chunks, embeddings)

# Guardar en disco (opcional)
db.save_local("faiss_ecoshop")


In [7]:
# ========================================
# 📌 4. Construcción del RAG
# ========================================
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

# LLM con Ollama
llm = Ollama(model="llama3.2:3b")

# Crear el pipeline RAG
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

print(" Pipeline RAG listo")


 Pipeline RAG listo


/tmp/ipython-input-2767297417.py:8: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2:3b")


In [10]:
# ========================================
# 📌 5. Pruebas del Sistema RAG
# ========================================

preguntas = [
   "¿Cuál es la política de devoluciones de EcoMarket?",

   "¿Cuánto tiempo tengo para devolver un producto que no he usado?",

   "¿Qué productos no aplican para devoluciones?",

   "¿En cuánto tiempo EcoMarket realiza el reembolso de un producto devuelto?",

   "¿Tienen disponibilidad del producto “Botella Reutilizable de Acero Inoxidable?",

   "¿Cuál es el precio actual del Cargador Solar Portátil?",

   "¿Cuántas unidades hay disponibles de Lámpara Solar para Jardín?",

   "¿Qué tipo de productos sostenibles ofrece EcoMarket en la categoría de Cuidado Personal?",

   "¿Puedo devolver un producto si olvidé la factura o comprobante de compra?",

   "¿Cómo puedo contactar al área de soporte técnico o atención al cliente de EcoMarket?",
]

for p in preguntas:
    print(f"\n🔹 Pregunta: {p}")
    respuesta = qa.run(p)
    print(f" Respuesta: {respuesta}")



🔹 Pregunta: ¿Cuál es la política de devoluciones de EcoMarket?
 Respuesta: La política de devoluciones de EcoMarket es la siguiente:

* Los clientes tienen 30 días para devolver un producto no usado.
* El reembolso se procesa en un plazo de 7 días hábiles.
* Los productos perecederos no aplican para devoluciones.
* Se requiere factura o comprobante de compra.

Para solicitar un reembolso, debe comunicarse con el equipo de soporte a través del correo electrónico soporte@ecomarket.com.

🔹 Pregunta: ¿Cuánto tiempo tengo para devolver un producto que no he usado?
 Respuesta: Tienes 30 días para devolver un producto no usado.

🔹 Pregunta: ¿Qué productos no aplican para devoluciones?
 Respuesta: Según la política de devoluciones de EcoMarket, los productos perecederos no aplican para devoluciones.

🔹 Pregunta: ¿En cuánto tiempo EcoMarket realiza el reembolso de un producto devuelto?
 Respuesta: Según la política de devoluciones de EcoMarket, se procesa el reembolso en un plazo de 7 días háb